<a href="https://colab.research.google.com/github/cb1711/Acceptance-Decision-Prediction/blob/master/decision_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
#!pip install reformer_pytorch

In [1]:
# try:
#   import apex
# except Exception:
#   ! git clone https://github.com/NVIDIA/apex.git
#   % cd apex
#   !pip install --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
#   %cd ..

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torch.optim as optim
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from transformers import BertConfig
from tqdm import tqdm_notebook
import torch.optim as optim
from tqdm.notebook import tqdm
import pickle
from torch.nn.modules.activation import MultiheadAttention


In [0]:
class Predictor(nn.Module):
  def __init__(self, max_len=512):
    super(Predictor, self).__init__()
    self.max_len = max_len
    self.bert = BertModel.from_pretrained('../scibert_scivocab_uncased', output_hidden_states=True)
    self.drop = nn.Dropout(p=0.35)
    self.fc3 = nn.Linear(768*max_len, 1)
  def forward(self, r1):
    _, pooled_out, hidden_states = self.bert(r1)
    mean_hidden = torch.mean(torch.stack(hidden_states,-1),-1)
    flat_out = mean_hidden.reshape((-1, 768*self.max_len))
    flat_out = self.drop(flat_out)  
    return F.sigmoid(self.fc3(flat_out))

In [3]:
!nvidia-smi
torch.cuda.manual_seed(10)

Sat Mar 28 19:25:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P8    12W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/dec_data

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/dec_data


In [0]:
def validate(model, tst, to):
  outs = []
  l = 0
  mxl = 0
  with torch.no_grad():
    for i in range(len(tst)):
      r1 = torch.LongTensor(tst[i]).unsqueeze(0).cuda()
      out = model(r1)
      loss = F.binary_cross_entropy(out, torch.FloatTensor([to[i]]).cuda())
      l+=loss.item()
      mxl = max(mxl, loss.item())
      outs.append(out.cpu())
  outs = [i.tolist()[0] for i in outs]
  outs = [int(i[0]>=0.5) for i in outs]
  print("max_loss is ", mxl)
  x = 0
  print("val_loss=" , l/len(to))
  for i in range(len(outs)):
    if outs[i]==int(to[i][0]):
      x=x+1
  return x/len(outs)

In [0]:
def train(year, fold):
  pred = Predictor(256).cuda()
  f = open('dec2018mkt.pkl','rb')
  tt = pickle.load(f)
  f = open('dec2017mkt.pkl','rb')
  tt = tt + pickle.load(f)
  f = open('dec2017mlt.pkl','rb')
  tt = tt + pickle.load(f)
  f = open('dec2018mlt.pkl','rb')
  tt = tt + pickle.load(f)
  f = open('dec2017mkt2.pkl','rb')
  tt = tt + pickle.load(f)
  f = open('dec2017mlt2.pkl','rb')
  tt = tt + pickle.load(f)
  f = open('dec2018mks.pkl','rb')
  ts = pickle.load(f)
  f = open('dec2017mks.pkl','rb')
  ts = ts + pickle.load(f)
  f = open('dec2017mls.pkl','rb')
  ts = ts + pickle.load(f)
  f = open('dec2018mls.pkl','rb')
  ts = ts + pickle.load(f)
  f = open('dec2017mks2.pkl','rb')
  ts = ts + pickle.load(f)
  f = open('dec2017mls2.pkl','rb')
  ts = ts + pickle.load(f)
  f = open('tests.pkl','rb')
  tst = pickle.load(f)
  f = open('testt.pkl','rb')
  ttt = pickle.load(f)
  ct = 0
  for param in pred.bert.parameters():
   if ct < 100:
     param.requires_grad = False
   ct += 1
  tt = [[i] for i in tt]
  ttt = [[i] for i in ttt]
  size = len(tt)
  fsize = size//20
  train_source = ts#[:(fsize*fold)] + ts[(fsize*(fold+1)):]
  train_target = tt#[:(fsize*fold)] + tt[(fsize*(fold+1)):]
  test_source = tst#[fsize*fold:fsize*(fold+1)]
  test_target = ttt#[fsize*fold:fsize*(fold+1)]
 
  tt = torch.FloatTensor(train_target)
  ts = train_source
  opti = optim.AdamW(pred.parameters(), lr = 1e-5)
  
  trainset = [(torch.LongTensor(ts[i]),tt[i]) for i in range(len(ts))]
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
  EPOCHS = 20
  val_max = 0

  pbar = tqdm()
  for epoch in tqdm(range(EPOCHS)):
    n = 0
    ls = 0 
    for data in tqdm(trainloader):
      r1,y = data
      pred.zero_grad()
      r1 = (torch.LongTensor(r1)).cuda()
      y = y.cuda()
      outputs = pred(r1)
      loss = F.binary_cross_entropy(outputs, y)
      loss.backward()
      l = loss.item()
      opti.step()
      ls = ls + l
      n=n+1
      pbar.set_description(str(l))
    print(ls/n)
    val_acc = validate(pred, test_source, test_target)
    print("val acc is ", val_acc, "\n")
    if val_acc >= val_max:
      print("Epoch ", epoch)
      print("test accuracy = ", val_acc, "\n")
     
      val_max = val_acc

In [0]:
train(2018,0)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



0.6633112869246958
max_loss is  1.8372042179107666
val_loss= 0.6879742153381047
val acc is  0.6052631578947368 

Epoch  0
test accuracy =  0.6052631578947368 




0.37431916621553624
max_loss is  3.111997604370117
val_loss= 0.546237460778732
val acc is  0.7368421052631579 

Epoch  1
test accuracy =  0.7368421052631579 




0.18166553647112926
max_loss is  4.26162576675415
val_loss= 0.5187236351990386
val acc is  0.8421052631578947 

Epoch  2
test accuracy =  0.8421052631578947 




0.10976163453112046
max_loss is  4.286404132843018
val_loss= 0.6005680728213567
val acc is  0.7631578947368421 



...


0.0417597218418168
max_loss is  6.96103572845459
val_loss= 0.8128593127755601
val acc is  0.7631578947368421 




0.03745348609382826
max_loss is  8.028995513916016
val_loss= 0.9102157824197906
val acc is  0.7631578947368421 




0.04502261032186304
max_loss is  8.845757484436035
val_loss= 1.1352540269081626
val acc is  0.631578947368421 




0.03735795497998792
max_loss is  6.16001558303833
val_loss= 0.773343916824639
val acc is  0.7631578947368421 




0.040431862350448766
max_loss is  8.11694049835205
val_loss= 0.905801925119273
val acc is  0.7368421052631579 

